In [7]:
import quimb.tensor as qtn
import numpy as np
from local_discriminant import disc_parts
from weights import ch_weight,sh_weight
from transformations import s_transform
from mps_generators import kraus_mps
import torch
import numba
import cupy as cp


In [8]:
L=40 #use even numbers
J=1
h=1.5
tol=1e-8
delta_t=.08
final_t=3.2

a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)

options={}
options['split_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel','max_bond':100}
options['tebd_opts']={'tol':tol,'order':4,'progbar':False}
options['comp_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel'}


In [9]:


I=np.eye(2)
X=np.array([[0,1],[1,0]])
Z=np.array([[1,0],[0,-1]])
ZZ=np.kron(Z,Z)
XI=np.kron(X,I)
IX=np.kron(I,X)

local_hamiltonians=[-J*ZZ+(h*IX+h*XI)/2]*(L-1)
local_hamiltonians[0]=-J*ZZ+h*XI+h/2*IX
local_hamiltonians[-1]=-J*ZZ+h/2*XI+h*IX

h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
h1=h*(np.kron(X,I)-np.kron(I,X))

ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False)
kraus=kraus_mps(Z,int(L/2),L)
weight_func=lambda x: sh_weight(x,a)

In [10]:
def func(lh,ps0,t):
    tebd=qtn.TEBD(ps0,lh,tol=tol,progbar=True,split_opts=options['split_opts'])
    tebd.update_to(t)
    return tebd


In [11]:
def change_backend(x):
    return torch.tensor(x,dtype=torch.complex128)

ham_tp.apply_to_arrays(change_backend)
kraus.apply_to_arrays(change_backend)



In [13]:
sca=s_transform(kraus,ch_weight,delta_t,num_steps,ham_tp,ham_tm,**options)

SystemError: CPUDispatcher(<function svd_truncated_numba at 0x7f1d8d9762a0>) returned a result with an exception set








kraus.apply_to_arrays(change_backend)
ham_tp.apply_to_arrays(change_backend)
ham_tm.apply_to_arrays(change_backend)
qtn.set_contract_backend('torch')


In [ ]:
res=s_transform(kraus,ch_weight,delta_t,num_steps,ham_tp,ham_tm,**options)

In [ ]:
res

[4,14.7],[6,21.9],[8,33.8],[10,41.3],[12,53.7],[14,62.6],[16,72.2],[18,82.4],[20,84,2],[22,102.6]

[4,7.3],[6,11.7],[8,13.5],[10,19.1],[12,22.8],[14,26.8],[16,30.3],[18,36.3],[20,34.6],[22,39.8]

In [ ]:
import matplotlib.pyplot as plt

# Data points
#data = [[4, 7.3], [6, 11.7], [8, 13.5], [10, 19.1], [12, 22.8], [14, 26.8], [16, 30.3], [18, 36.3], [20, 34.6], [22, 39.8]]
data=[[4,14.7],[6,21.9],[8,33.8],[10,41.3],[12,53.7],[14,62.6],[16,72.2],[18,82.4],[20,84,2],[22,102.6]]
# Separate the data into x and y coordinates
x, y = zip(*data)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x, y, marker='o', linestyle='-', color='b')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Data Points Plot')
plt.grid(True)
plt.show()